### 主成分分析
参考文献:<br>
> 井出剛. 2015. 入門機械学習による異常検知. コロナ社. pp.123-139. <br>
> MacGregor JF, Kourti T. 1995. Statistical process control of multivariate processes. Control Eng. Practice. 3: 403-414.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from typing import Union, Literal, Annotated

#### PCA法の特徴
1. 各変数の情報からうまく基底をつくりだすことにより、多次元の有する情報を低次元に落とし込むことが可能<br>
2. 上記より、不要な変数の寄与を取り除くことができる
3. 上記より、多次元の情報を直観的に理解しやすくすることができる

&nbsp;

ただし、以下点については注意が必要である。
1. 変数の線形和であらわされた新たな変数（主成分）は、必ずしも解釈が容易ではない
2. 各変数の寄与は一般に不明である ※1
3. 閾値設定は経験的なものとなる ※2

&nbsp;

**<注釈>** <br>
※1 各変数の寄与の算出はできない、というわけではないらしく、例えば以下には算出方法の記載が存在する <br>
> https://www.jmp.com/support/help/ja/17.2/index.shtml#page/jmp/statistical-details-for-contributions.shtml# <br>

※2 井出には記載がないものの、ベータ分布やF分布で閾値決めをしている例も存在。<br>
> https://www.jmp.com/support/help/ja/17.2/index.shtml#page/jmp/statistical-details-for-limits.shtml#